In [15]:
import os

In [16]:
%pwd

'd:\\A_Category\\iNeuron\\End-To-End-NLP-Project-News-Article-Sorting'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\A_Category\\iNeuron\\End-To-End-NLP-Project-News-Article-Sorting'

In [17]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [18]:
from ArticleSorting.constants import *
from ArticleSorting.utils.common import read_yaml, create_directories

In [19]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) ->DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path= config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config


In [20]:
import os
from ArticleSorting.logging import logger

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import Dataset

import torch
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np


In [34]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
        

    def encode_categories(self):
        df = pd.read_csv(self.config.data_path)
        df['encoded_label'] = df['Category'].astype('category').cat.codes

        '''Spliting the Data
        # Training dataset
        #train_data = df.sample(frac=0.8, random_state=42)
        # Testing dataset
        #test_data = df.drop(train_data.index)
        '''

        # Split train , test and validatio dataset
        np.random.seed(112)
        df_train, df_test, df_val = np.split(df.sample(frac=1, random_state=35),
                                     [int(0.7*len(df)), int(0.8*len(df))])

        print(len(df_train), len(df_test), len(df_val)) 
        

        # Convert pyhton dataframe to Hugging Face arrow dataset
        hg_train_data = Dataset.from_pandas(df_train)
        hg_test_data = Dataset.from_pandas(df_test)
        hg_val_data = Dataset.from_pandas(df_val)
        print(hg_train_data, hg_test_data, hg_val_data)
        return hg_train_data, hg_test_data, hg_val_data
   


    def tokenize_dataset(self, data):
        return self.tokenizer(data["Text"],
                     max_length=512,
                     truncation=True,
                     padding="max_length")
    


    def convert(self):
        hg_train_data, hg_test_data, hg_val_data = self.encode_categories()

        # Tokenize the dataset
        dataset_train = hg_train_data.map(self.tokenize_dataset)
        dataset_test = hg_test_data.map(self.tokenize_dataset)
        dataset_val = hg_val_data.map(self.tokenize_dataset)

        
        # Remove the review and index columns because it will not be used in the model
        dataset_train = dataset_train.remove_columns(["ArticleId", "Text", "Category", "__index_level_0__"])
        dataset_test = dataset_test.remove_columns(["ArticleId", "Text", "Category", "__index_level_0__"])
        dataset_val = dataset_val.remove_columns(["ArticleId", "Text", "Category", "__index_level_0__"])

        # Rename label to labels because the model expects the name labels
        dataset_train = dataset_train.rename_column("encoded_label", "labels")
        dataset_test = dataset_test.rename_column("encoded_label", "labels")
        dataset_val = dataset_val.rename_column("encoded_label", "labels")


        # Change the format to PyTorch tensors
        dataset_train.set_format("torch")
        dataset_test.set_format("torch")
        dataset_val.set_format("torch")

        # Take a look at the data
        print(dataset_train)
        print(dataset_test)
        print(dataset_val)

        dataset_train.save_to_disk(os.path.join(self.config.root_dir,"Train BBC dataset"))
        dataset_test.save_to_disk(os.path.join(self.config.root_dir,"Test BBC dataset"))
        dataset_val.save_to_disk(os.path.join(self.config.root_dir,"Validation BBC dataset"))


In [35]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
  
except Exception as e:
    raise e

[2023-11-21 00:25:20,240:  INFO: common: yaml file:config\config.yaml loaded successfully]
[2023-11-21 00:25:20,245:  INFO: common: yaml file:params.yaml loaded successfully]
[2023-11-21 00:25:20,247:  INFO: common: created directory at : artifacts]
[2023-11-21 00:25:20,250:  INFO: common: created directory at : artifacts/data_transformation]
1043 149 298
Dataset({
    features: ['ArticleId', 'Text', 'Category', 'encoded_label', '__index_level_0__'],
    num_rows: 1043
}) Dataset({
    features: ['ArticleId', 'Text', 'Category', 'encoded_label', '__index_level_0__'],
    num_rows: 149
}) Dataset({
    features: ['ArticleId', 'Text', 'Category', 'encoded_label', '__index_level_0__'],
    num_rows: 298
})


Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1043
})
Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 149
})
Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 298
})
